In [2]:
from __future__ import print_function 

#%load_ext autoreload
#%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist,mnist_m
from data.tasks import *
from data.xray import * # temporary
from data.label_shift import *
from experiments.models import *
from experiments.training import *
from util.kl import *
from util.misc import *


#reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()


    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
    
reset_keras()

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

# Load data & Train

In [3]:

#alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
alphas=[0.3]#,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]#,0.3,0.5,0.7,0.9]
#epsilons=[0,0.1]#,0.01,0.001]
architectures=["resnet"]
tasks=[7]
seeds=[1]
image_size=64
batch_size=32
num_priors=1
from data.xray import *
import time
for t in tasks:
    for arch in architectures:
        for alpha in alphas:
            for seed in seeds:
                if t==7:
                        source_generator, target_generator=load_task(t,architecture=arch,binary=True,image_size=image_size,batch_size=batch_size)
                else:
                        x_source,y_source,x_target,y_target,_, _=load_task(task=t,architecture=arch,binary=True)

                        
                        sys.exit(-1)

                if alpha==0:
                    pass
                else:
                    if t==7:
                        prior_generator, bound_generator, target_generator=load_task(task=t,alpha=alpha,architecture=arch,binary=True,image_size=image_size,seed=seed) ## create bound/prior split here as own generators
                        print(len(bound_generator.y))
                        print(len(target_generator.y))
                        sys.exit(-1)
                        w_a=train_prior(alpha,num_priors,generator=prior_generator,val_generator=target_generator,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)
                    else:
                        print("Alpha is:"+str(alpha))
                        #x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=seed)
                        w_a=train_prior(alpha,num_priors,x_train=x_source,y_train=y_source,x_target=x_target,y_target=y_target,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)
#                         project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"
#                         result_path=project_folder+"results/"+"task"+str(t)+"/Binary/"+str(arch)
                #for epsilon in epsilons:
                if t==7:
                    if alpha!=0:
                        source_generator, target_generator=load_task(t,architecture=arch,binary=True,image_size=image_size)
                    print(len(source_generator))
                    w_s=train_posterior(alpha,generator=source_generator,val_generator=target_generator,prior_weights=None,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)
                else:
                    w_s=train_posterior(alpha,x_train=x_source,y_train=y_source,prior_weights=None,x_test=x_source,y_test=y_source,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)



            

4.019297905352987
3.9835700132100396
length of chestxray in source 22424
amount of chest in source and target 4.0
mean of weights in source, should be 1:  1.0000000000000002
4.039824732229795
3.9663485272338326
length of chestxray in source 22424
amount of chest in source and target 4.0
mean of weights in source, should be 1:  1.0
172250
89696


SystemExit: -1

/apps/Arch/software/IPython/7.18.1-GCCcore-10.2.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#model.evaluate(target_generator,steps=target_generator.steps, workers=16)